In [1]:
import torch
from torch import nn
from dataclasses import dataclass
from src.utils.common import *
from src import logger
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, VANILLA_FILE_PATH
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from src.features.image_transformation import ImageTransformation
from src.config.configuration import ConfigurationManager
from src.models.vanilla.model import VanillaModel
from torch import optim
from importlib import import_module
import os
import mlflow

In [2]:
os.chdir("../")
print(os.getcwd())

/Users/goldyrana/mess/deep_learning/projects/blood_group_detection


In [3]:
from src.config.configuration import VanillaModelConfig, ImageTransformationConfig

In [3]:
@dataclass
class ModelTrainingConfig:
    epochs: int
    basemodule: str
    submodule: str
    modelclass: str
    optimizer: str
    optimizer_args: dict
    criterion: str

class ConfigurationManager:
    def __init__(self):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.params=read_yaml(PARAMS_FILE_PATH)
        self.vanilla=read_yaml(VANILLA_FILE_PATH) # get vanilla params

    def get_model_training_config(self):
        params=ModelTrainingConfig(
            epochs=self.params.training.epochs,
            basemodule=self.params.train_model.basemodule,
            submodule=self.params.train_model.submodule,
            modelclass=self.params.train_model.modelclass,
            optimizer=self.params.train_model.optimizer,
            optimizer_args=self.params.train_model.optimizer_args,
            criterion=self.params.train_model.criterion
        )
        return params
    
    def get_image_transformation_config(self)->ImageTransformationConfig:
        params=ImageTransformationConfig(
            image_shape=[self.params.image_transformation.height,
                        self.params.image_transformation.width],

            mean=self.params.image_transformation.mean,
            std=self.params.image_transformation.std,
            # Directory params

            train_path=self.config.data.train_path,
            batch_size=self.params.image_transformation.batch_size,
            shuffle=self.params.image_transformation.shuffle,
            num_workers=self.params.image_transformation.workers
        )
        return params
     
    def get_architecture_config(self) -> VanillaModelConfig:
        if self.params.train_model.modelclass == "VanillaModel":
            params=VanillaModelConfig(
                # Getting params from params.yaml
                image_height=self.params.image_params.height,
                image_width=self.params.image_params.width,

                # Getting vanilla params from vanilla_params.yaml
                layer_1=self.vanilla.layers.first, # access vanilla architecture layer 1 params
                layer_2=self.vanilla.layers.second # access vanilla architecture layer 2 params
            )
            return params


class ModelTraining:
    def __init__(self, config):
        self.config=config
        self.device= str("gpu" if torch.cuda.is_available() else "cpu")
    
    def get_optimizer(self):
        module = import_module("torch.optim")
        optimizer = getattr(module, self.config.optimizer)
        return optimizer
    
    def get_criterion(self):
        module = import_module("torch.nn")
        criterion = getattr(module, self.config.criterion)
        return criterion
    
    def get_model(self, basemodule= None, 
                        submodule= None,
                        modelclass= None) -> type:
        """
        Args:
            basemodule(str): Directory where models are present
            submodule(str): name of model
            model_class(str): Class name of model
        Return: 
            object of t
        """
        basemodule= basemodule or self.config.basemodule 
        submodule= submodule or self.config.submodule
        modelclass= modelclass or self.config.modelclass

        module = import_module(f"{basemodule}.{submodule}")
        model = getattr(module, modelclass)
        return model
    
    def get_model_training_attributes(self, model=None, optimizer=None, criterion=None):
        """
        Returns the model attributes which is used to build the model
        Args:
            model(type): model which needs to be trained
            optimizer(type): model's optimizer
            criterion(type): model's criterion
        Return:
            Tuple: (model, optimizer, criterion ) 
        """
        try:
            optimizer = self.get_optimizer() or optimizer
            criterion = self.get_criterion() or criterion
            model = self.get_model() or criterion
            return model, optimizer, criterion
        except Exception as e:
            logger.error(e)
            raise e
    
    def train(self, train_loader, model_params, model=None,  criterion=None, optimizer=None):
               
        try:
            model = model(model_params)
            optimizer = optimizer(model.parameters(), **self.config.optimizer_args)
            criterion = criterion()
        except Exception as e:
            logger.error("Error occured while initalizing model")
            raise e
        
        for epoch in range(self.config.epochs):
            model.train()

            for images, labels in train_loader:
                images, labels = images.to(self.device), labels.to(self.device)

                # forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                print(f'Epoch [{epoch+1}/{2}], Loss: {loss.item():.4f}')



    

                



NameError: name 'ImageTransformationConfig' is not defined

In [16]:
# from src.config.configuration import ConfigurationManager

if __name__ == "__main__":
    config_manager = ConfigurationManager()
    # config_params = config_manager.get_model_training_config()

    # Model architecture get_vanilla_architecture_config
    vanilla_params = config_manager.get_architecture_config()
    try:
        model = VanillaModel(vanilla_params)
        # optimizer = optim.Adam(model.parameters(), lr = 0.01)
        # criterion = nn.CrossEntropyLoss()
        logger.info(f"Initializing Model: {model.__name__()}")
    
    except Exception as e:
        logger.error("Error occured while initalizing model")
        raise e

    # Data Loader
    image_transformation_params = config_manager.get_image_transformation_config()
    image_transformation = ImageTransformation(image_transformation_params)

    train_transformer = image_transformation.get_image_transformer(data="train") # perform image_transformation on training dataset
    train_loader = image_transformation.get_data_loader(config_manager.config.data.train_path, 
                                                        train_transformer)


    # Model Training
    # try:
    #     logger.info("Model Training Intialized")
    #     model_training_params = config_manager.get_model_training_config()
    #     trainer = ModelTraining(model_training_params)
    
    #     trainer.train(model, train_loader, criterion, optimizer)
    # except Exception as e:
    #     logger.error("Error occured in model training")

    
    

2025-01-15 17:54:00,484 - root - INFO - Yaml read successfully from config/config.yaml
2025-01-15 17:54:00,488 - root - INFO - Yaml read successfully from params.yaml
2025-01-15 17:54:00,490 - root - INFO - Yaml read successfully from src/models/vanilla/vanilla_params.yaml
2025-01-15 17:54:00,890 - root - INFO - Initializing Model: Vanilla
2025-01-15 17:54:00,891 - root - INFO - Initializing transformation on train data


In [17]:
try:
    logger.info("Model Training Intialized")
    model_training_params = config_manager.get_model_training_config()
    trainer = ModelTraining(model_training_params)
    model_params = config_manager.get_architecture_config()
    model, optimizer, criterion  = trainer.get_model_training_attributes()
    trainer.train(train_loader, model_params, model, criterion, optimizer)
except Exception as e:
    logger.error("Error occured in model training")
    raise e


2025-01-15 17:54:00,972 - root - INFO - Model Training Intialized


Epoch [1/2], Loss: 2.0964
Epoch [1/2], Loss: 829.0962
Epoch [1/2], Loss: 396.3242
Epoch [1/2], Loss: 2732.3208
Epoch [1/2], Loss: 78.0780
Epoch [1/2], Loss: 3.6699
Epoch [1/2], Loss: 2.0794
Epoch [1/2], Loss: 2.0794
Epoch [1/2], Loss: 2.0794
Epoch [1/2], Loss: 2.0794
Epoch [1/2], Loss: 2.0794
Epoch [1/2], Loss: 2.0794
Epoch [1/2], Loss: 2.0794
Epoch [1/2], Loss: 2.0794
Epoch [1/2], Loss: 2.0794
Epoch [1/2], Loss: 2.0794


In [7]:
model_params

VanillaModelConfig(image_height=100, image_width=96, layer_1=ConfigBox({'in_channels': 3, 'out_channels': 32, 'kernel_size': [3, 3], 'stride': 1, 'padding': 1}), layer_2=ConfigBox({'in_channels': 32, 'out_channels': 64, 'kernel_size': [3, 3], 'stride': 1, 'padding': 1}))

In [20]:
trainer.get_criterion(), trainer.get_optimizer()
trainer.get_model()

src.models.vanilla.model.VanillaModel

In [4]:
from src.constants import (CONFIG_FILE_PATH, PARAMS_FILE_PATH,
                        VANILLA_FILE_PATH)
from src.features.image_transformation import ImageTransformation
from src.config.configuration import ConfigurationManager
from src.models.vanilla.model import VanillaModel
from src import logger
from tqdm import tqdm

from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch import optim

from importlib import import_module
import torch
import mlflow
import os


class ModelTraining:
    def __init__(self, config):
        self.config=config
        self.device= str("gpu" if torch.cuda.is_available() else "cpu")
    
    def get_optimizer(self):
        module = import_module("torch.optim")
        optimizer = getattr(module, self.config.optimizer)
        return optimizer
    
    def get_criterion(self):
        module = import_module("torch.nn")
        criterion = getattr(module, self.config.criterion)
        return criterion
    
    def get_model(self, basemodule= None, 
                        submodule= None,
                        modelclass= None) -> type:
        """
        Args:
            basemodule(str): Directory where models are present
            submodule(str): name of model
            model_class(str): Class name of model
        Return: 
            object of t
        """
        basemodule= basemodule or self.config.basemodule 
        submodule= submodule or self.config.submodule
        modelclass= modelclass or self.config.modelclass

        module = import_module(f"{basemodule}.{submodule}")
        model = getattr(module, modelclass)
        return model
    
    def get_model_training_attributes(self, model=None, optimizer=None, criterion=None):
        """
        Returns the model attributes which is used to build the model
        Args:
            model(type): model which needs to be trained
            optimizer(type): model's optimizer
            criterion(type): model's criterion
        Return:
            Tuple: (model, optimizer, criterion ) 
        """
        try:
            optimizer = self.get_optimizer() or optimizer
            criterion = self.get_criterion() or criterion
            model = self.get_model() or criterion
            return model, optimizer, criterion
        except Exception as e:
            logger.error(e)
            raise e
    
    def train(self, train_loader, model_params, model=None,  criterion=None, optimizer=None):
               
        try:

            model = model(model_params)
            optimizer = optimizer(model.parameters(), **self.config.optimizer_args)
            criterion = criterion()
            logger.info("Model Attributes initalized")
        except Exception as e:
            logger.error("Error occured while initalizing model")
            raise e
        
        print("Starting training loop...")
        for epoch in tqdm(range(self.config.epochs), desc="Training Epochs"):
            model.train()
            
            if len(train_loader) > 0:
                for images, labels in train_loader:
                    print("Batch size:", images.size())
                    break
            else:
                print("Data loader is empty. Check your dataset and path.")
            
            for images, labels in train_loader:
                images, labels = images.to(self.device), labels.to(self.device)

                # forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                logger.info(f'Epoch [{epoch+1}/{2}], Loss: {loss.item():.4f}')
                print(f'Epoch [{epoch+1}/{2}], Loss: {loss.item():.4f}')


if __name__ == "__main__":
    config_manager = ConfigurationManager()
    config_params = config_manager
    


    # Data Loader
    image_transformation_params = config_manager.get_image_transformation_config()
    image_transformation = ImageTransformation(image_transformation_params)

    train_transformer = image_transformation.get_image_transformer(data="train") # perform image_transformation on training dataset
    train_loader = image_transformation.get_data_loader(config_manager.config.data.train_path, 
                                                        train_transformer)
    # # Model Training
    try:
        logger.info("Model Training Intialized")
        model_training_params = config_manager.get_model_training_config()
        trainer = ModelTraining(model_training_params)

        # Model Architecture
        model_params = config_manager.get_architecture_config()
        model, optimizer, criterion  = trainer.get_model_training_attributes()
        logger.info(f"Initializing Model: {model.__name__}")
        
        # Training Loop
        trainer.train(train_loader, model_params, model, criterion, optimizer)
    
    except Exception as e:
        logger.error("Error occured in model training")
        raise e

2025-01-15 18:34:05,134 - root - INFO - Yaml read successfully from config/config.yaml
2025-01-15 18:34:05,137 - root - INFO - Yaml read successfully from params.yaml
2025-01-15 18:34:05,138 - root - INFO - Yaml read successfully from src/models/vanilla/vanilla_params.yaml
2025-01-15 18:34:05,139 - root - INFO - Initializing transformation on train data
2025-01-15 18:34:05,147 - root - INFO - Model Training Intialized
2025-01-15 18:34:05,148 - root - INFO - Initializing Model: VanillaModel
2025-01-15 18:34:05,505 - root - INFO - Model Attributes initalized


Starting training loop...


Training Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

Batch size: torch.Size([2, 3, 100, 96])
